In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from models import Autoencoder, EncoderStack
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from tensorflow.keras.utils import to_categorical

# Load and check data

In [2]:
FILE_NAME = "GEO_data_batch_corr_final"
FILE_CLASSES = "GBM_class"

N_LAYERS = 3
N_NODES = [2000, 1000, 500]
DROPOUT = [0.1]
BATCH_SIZE = 3
EPOCHS = 10
TEST_RATIO = 0.15

assert len(N_NODES) == N_LAYERS or len(N_NODES) == 1
assert len(DROPOUT) == N_LAYERS or len(DROPOUT) == 1

In [9]:
dataframe = pd.read_pickle('data/pd/'+FILE_NAME)
print("Loaded {} samples with {} features.".format(dataframe.shape[0], dataframe.shape[1]))
data = dataframe.values
data = normalize(data)

classes = pd.read_csv('data/pd/class.csv', header=None, index_col=0).values
classes = to_categorical(classes)

x_train, x_test, y_train, y_test = train_test_split(data, classes, test_size=TEST_RATIO)
print("Keeping {} of samples for training: {} training, {} testing.".format(TEST_RATIO, x_train.shape[0], x_test.shape[0]))

Loaded 954 samples with 6785 features.
Keeping 0.15 of samples for training: 810 training, 144 testing.


# Train encoder layers

In [10]:
encoder_models = []

x_train_out, x_test_out = x_train, x_test
for idx, num_hidden in enumerate(N_NODES):
    print("\n##################################################################")
    print("Training layer {} with {} hidden nodes..\n".format(idx, num_hidden))
    encoder = Autoencoder(x_train_out.shape[1], num_hidden, "output/")
    recon_mse = encoder.fit(x_train_out, x_test_out, batch_size=BATCH_SIZE, num_epochs=EPOCHS)
    x_train_out = encoder.encoder_model.predict(x_train_out)
    x_test_out = encoder.encoder_model.predict(x_test_out)
    print("\nTraining losss: ", recon_mse[0])
    print("\nTesting loss: ", recon_mse[1])
    encoder_models.append(encoder)


##################################################################
Training layer 0 with 2000 hidden nodes..

Epoch 1/10
270/270 [==============================] - 66s 245ms/step - loss: 0.0376 - val_loss: 3.4984e-04
Epoch 2/10
270/270 [==============================] - 65s 241ms/step - loss: 1.0122e-04 - val_loss: 3.1444e-05
Epoch 3/10
270/270 [==============================] - 66s 244ms/step - loss: 2.2238e-05 - val_loss: 1.2747e-05
Epoch 4/10
270/270 [==============================] - 66s 246ms/step - loss: 1.1555e-05 - val_loss: 9.3970e-06
Epoch 5/10
270/270 [==============================] - 67s 247ms/step - loss: 8.1724e-06 - val_loss: 7.7891e-06
Epoch 6/10
270/270 [==============================] - 68s 251ms/step - loss: 6.2940e-06 - val_loss: 8.3466e-06
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: output/20200727-051706/autoencoder-2000/assets

Training losss:  8.476895e-06

Testing loss:  8.457224e-06

###

# Train encoder stack

In [11]:
model = EncoderStack(encoder_models, 'output/')

print("\n##################################################################")
print("Training layer {} with {} hidden nodes..\n".format(idx, num_hidden))
loss_train, loss_test = model.fit(x_train, y_train, x_test, y_test, batch_size=BATCH_SIZE, num_epochs=EPOCHS)

print("\nTraining losss: ", loss_train)
print("\nTesting loss: ", loss_test)


##################################################################
Training layer 2 with 500 hidden nodes..

Epoch 1/10
270/270 [==============================] - 46s 172ms/step - loss: 0.0912 - val_loss: 0.0657
Epoch 2/10
270/270 [==============================] - 50s 184ms/step - loss: 0.0893 - val_loss: 0.0655
Epoch 3/10
270/270 [==============================] - 53s 195ms/step - loss: 0.0844 - val_loss: 0.0565
Epoch 4/10
270/270 [==============================] - 50s 184ms/step - loss: 0.0685 - val_loss: 0.0418
Epoch 5/10
270/270 [==============================] - 49s 183ms/step - loss: 0.0541 - val_loss: 0.0389
Epoch 6/10
270/270 [==============================] - 46s 170ms/step - loss: 0.0467 - val_loss: 0.0351
Epoch 7/10
270/270 [==============================] - 47s 173ms/step - loss: 0.0436 - val_loss: 0.0416
Epoch 00007: early stopping
INFO:tensorflow:Assets written to: output/20200727-053055/encoder_stack/assets

Training losss:  0.27949008

Testing loss:  0.2811888
